In [1]:
import acquire

In [3]:
df = acquire.get_sql_zillow()

In [4]:
df.propertylandusedesc.value_counts()

Single Family Residential                     52442
Condominium                                   19342
Duplex (2 Units, Any Combination)              2021
Planned Unit Development                       1947
Quadruplex (4 Units, Any Combination)           729
Triplex (3 Units, Any Combination)              539
Cluster Home                                    335
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       59
Residential General                              42
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: propertylandusedesc, dtype: int64

In [10]:
df = df [df.transactiondate.str.startswith("2017")]

In [11]:
df = df.sort_values("transactiondate", ascending=False).drop_duplicates("parcelid")

In [12]:
df = df [df.propertylandusedesc == "Single Family Residential"]

In [13]:
df.unitcnt.value_counts(dropna=False)

1.0    33734
NaN    18558
2.0       27
3.0        1
Name: unitcnt, dtype: int64